In [ ]:
# TP Segmentation d'Images avec UNet

# Étape 1 : Préparation de l'environnement
!pip install tensorflow keras numpy matplotlib opencv-python-headless scikit-learn

# Importations des bibliothèques
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

# Configuration de TensorFlow
print("Version TensorFlow :", tf.__version__)
print("GPU disponible :", tf.test.is_gpu_available())

# Étape 2 : Génération de données synthétiques (remplacer par votre propre dataset)
def generate_synthetic_dataset(num_images=100, img_height=128, img_width=128):
    # Génération d'images et masques synthétiques
    images = np.random.rand(num_images, img_height, img_width, 3)
    masks = np.zeros((num_images, img_height, img_width, 1), dtype=np.float32)

    # Création de masques simples (cercle au centre)
    for i in range(num_images):
        center_y, center_x = img_height // 2, img_width // 2
        for y in range(img_height):
            for x in range(img_width):
                if (x - center_x)**2 + (y - center_y)**2 < (min(img_height, img_width) // 4)**2:
                    masks[i, y, x, 0] = 1.0

    return images, masks

# Génération du dataset
images, masks = generate_synthetic_dataset(num_images=200)

# Étape 3 : Préparation des données
# Division train, validation, test
X_train, X_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Formes des données :")
print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

# Étape 4 : Prétraitement des données
# Normalisation
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Étape 5 : Construction du modèle UNet
def unet_model(input_size=(128, 128, 3), num_classes=1):
    inputs = keras.layers.Input(input_size)

    # Contracting Path
    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)

    # Expanding Path
    up5 = keras.layers.UpSampling2D(size=(2, 2))(conv4)
    up5 = keras.layers.Concatenate()([up5, conv3])
    conv5 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(up5)
    conv5 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv5)

    up6 = keras.layers.UpSampling2D(size=(2, 2))(conv5)
    up6 = keras.layers.Concatenate()([up6, conv2])
    conv6 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(up6)
    conv6 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv6)

    up7 = keras.layers.UpSampling2D(size=(2, 2))(conv6)
    up7 = keras.layers.Concatenate()([up7, conv1])
    conv7 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(up7)
    conv7 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv7)

    # Output
    outputs = keras.layers.Conv2D(num_classes, 1, activation='sigmoid')(conv7)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Création du modèle
model = unet_model()
model.summary()

# Étape 6 : Compilation du modèle
def dice_coef(y_true, y_pred, smooth=1):
    intersection = keras.backend.sum(y_true * y_pred, axis=[1,2,3])
    union = keras.backend.sum(y_true, axis=[1,2,3]) + keras.backend.sum(y_pred, axis=[1,2,3])
    return keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# Compilation du modèle
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss=dice_coef_loss,
              metrics=[dice_coef, 'accuracy'])

# Étape 7 : Entraînement du modèle
# Data Augmentation
data_gen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Entraînement
history = model.fit(
    data_gen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val),
    steps_per_epoch=len(X_train) // 32
)

# Étape 8 : Évaluation et Prédiction
# Évaluation sur l'ensemble de test
results = model.evaluate(X_test, y_test)
print("Test Loss:", results[0])
print("Test Dice Coefficient:", results[1])
print("Test Accuracy:", results[2])

# Visualisation des prédictions
def plot_predictions(model, X_test, y_test, num_images=5):
    plt.figure(figsize=(15, 5*num_images))
    for i in range(num_images):
        # Image originale
        plt.subplot(num_images, 3, i*3 + 1)
        plt.imshow(X_test[i])
        plt.title("Image Originale")
        plt.axis('off')

        # Masque réel
        plt.subplot(num_images, 3, i*3 + 2)
        plt.imshow(y_test[i].squeeze(), cmap='gray')
        plt.title("Masque Réel")
        plt.axis('off')

        # Prédiction
        pred = model.predict(X_test[i:i+1])[0]
        plt.subplot(num_images, 3, i*3 + 3)
        plt.imshow(pred.squeeze(), cmap='gray')
        plt.title("Masque Prédit")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

plot_predictions(model, X_test, y_test)

# Étape 9 : Courbes d'apprentissage
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['dice_coef'], label='Train Dice Coef')
plt.plot(history.history['val_dice_coef'], label='Validation Dice Coef')
plt.title('Dice Coefficient')
plt.legend()

plt.tight_layout()
plt.show()

Version TensorFlow : 2.17.1
GPU disponible : False


KeyboardInterrupt: 